In [1]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\4.2.1\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [3]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    // names are in format [signal, strategy]
    let strategy = summary.StrategyName.Substring(summary.StrategyName.IndexOf(",") + 2)
    printfn "%i trades, %.2f%% win pct, %.2f%% avg gain, %.2f%% avg loss, %.2f avg gain/loss, %.2f EV" 
        strategy summary.Total (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV

    match includeChart with
    | true ->
        let chart = 
            Chart.Histogram(
                X = summary.Gains,
                NBinsX = 40,
                Name = "Gain Distribution"
            )

        chart.Display() |> ignore
    | false -> ()

In [50]:
let filterNamePairs = [
        ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
        ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
        ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
        ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
        ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
        ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
    ]

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    let strategy = filteredTrades |> Seq.head |> fun x -> x.Strategy
                    let name = $"{filterName}, {strategy}"
                    studies.Types.TradeSummary.create name filteredTrades
            )
    )
    

let describeOutcomes includeChart tradeSummaries =
    
    let groupedBySignal = tradeSummaries |> Seq.groupBy (fun (s:studies.Types.TradeSummary) -> s.StrategyName.Split(",")[0])
    
    groupedBySignal
    |> Seq.iter (
        fun (signal,summaries) ->
            
            printfn $"{signal} [{summaries |> Seq.head |> fun x -> x.Total}]"
            
            summaries |> Seq.iter (fun summary -> outputSummary includeChart summary)
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
    
    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        tradeSummaries
        |> Seq.sortByDescending sortFunction
        |> Seq.take numberOfTopRecords
        |> Seq.iter (fun summary ->
            printfn $"{summary.StrategyName}: {sortFunction summary}"
        )
        printfn ""
        printfn ""
    )

In [5]:
let studiesDirectory = "d:\\studies"
let signalPath = "d:\\studies\\02_export_date_ticker_screenerid_gap.csv"
let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let runStrategies strategies = async {
    let! signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc
    
    return! studies.Trading.runTrades signalsWithPrices strategies
}

In [59]:
let strategies = 
        [
            studies.TradingStrategies.buyAndHoldStrategy None
            studies.TradingStrategies.buyAndHoldStrategy (Some 5)
            studies.TradingStrategies.buyAndHoldStrategy (Some 15)
            studies.TradingStrategies.buyAndHoldStrategy (Some 30)
            studies.TradingStrategies.buyAndHoldStrategy (Some 60)
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false None 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 5) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 15) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 30) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 60) 5m
        ]

let outcomes = 
    strategies
    |> runStrategies
    |> Async.RunSynchronously

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

printfn "Generating trade summaries..."
let tradeSummaries = outcomes |> createTradeSummaries |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Executing trades...
Finished running trades, generated:
228480 trade outcomes
Generating trade summaries
60 summaries generated


In [57]:
let outcomeDescription (x:studies.Types.TradeOutcomeOutput.Row) =
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened.ToShortDateString()} @ {x.OpenPrice}, {x.Closed.ToShortDateString()} @ {x.ClosePrice}: profit of {x.PercentGain}"
    
let random = System.Random()

outcomes
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription


ANF Buy and Hold: 8/17/2022 @ 21.05, 11/14/2023 @ 69.12: profit of 228.36104513064133016627078385
SJT Buy and Hold 60 bars with Stop Loss of 5.00%: 4/19/2022 @ 11.02, 4/19/2022 @ 10.29: profit of -6.6243194192377495462794918300
CDW Buy and Hold 30 bars with Stop Loss of 5.00%: 8/3/2023 @ 195.56, 9/15/2023 @ 207.0: profit of 5.8498670484761709961137246900
CEG Buy and Hold: 9/15/2023 @ 112.46, 11/14/2023 @ 124.844: profit of 11.011915347679174817712964610
ATEC Buy and Hold 30 bars with Stop Loss of 5.00%: 7/14/2023 @ 18.75, 7/20/2023 @ 17.77: profit of -5.2266666666666666666666666700
UBER Buy and Hold with Stop Loss of 5.00%: 5/3/2023 @ 36.75, 11/14/2023 @ 54.22: profit of 47.537414965986394557823129250
VERU Buy and Hold: 8/12/2022 @ 15.78, 11/14/2023 @ 0.971051: profit of -93.84631812420785804816223067
HCM Buy and Hold: 10/13/2023 @ 17.9, 11/14/2023 @ 19.11: profit of 6.7597765363128491620111731800
ZLAB Buy and Hold 5 bars with Stop Loss of 5.00%: 11/7/2023 @ 27.76, 11/14/2023 @ 28.545:

In [53]:
// inspect what's max gain 
outcomes
|> Seq.filter (fun x -> x.Strategy.Contains("with Stop Loss"))
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription


SMCI Buy and Hold with Stop Loss of 5.00%: 7/22/2022 @ 50.92, 11/14/2023 @ 282.25: profit of 454.30086410054988216810683425


In [9]:
// investigating what happened with the trade and why it went as it went
let signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc |> Async.RunSynchronously

let tickerOfInterest = "MLTX"

let signal, prices = 
    signalsWithPrices
    |> Seq.filter (fun (signal, prices) -> signal.Ticker = tickerOfInterest)
    |> Seq.head
    
let strategy = studies.TradingStrategies.buyAndHoldStrategyWithStopLoss true (Some 5) 5m

(signal, prices) |> strategy |> outcomeDescription

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Open day for MLTX on 12/28/2022 12:00:00 AM is 12/29/2022 12:00:00 AM +00:00 with open day index of 183
Close day for MLTX on 12/28/2022 12:00:00 AM is based on Some 5 number of bars to hold and is 1/6/2023 12:00:00 AM +00:00 with close day index of 188
Close bar for MLTX on 12/28/2022 12:00:00 AM is 1/6/2023 12:00:00 AM +00:00 because close day reached
MLTX Buy and Hold 5 bars with Stop Loss of 5.00%: 12/29/2022 @ 10.22, 1/6/2023 @ 12.52: profit of 22.504892367906066536203522500


In [61]:
tradeSummaries |> describeOutcomes false

All [22848]
43.08%, 29.77% 	 -30.46% 	 0.98 avg gain/loss, -4.52 EV All, Buy and Hold
50.28%, 7.40% 	 -6.87% 	 1.08 avg gain/loss, 0.30 EV All, Buy and Hold 5 bars
48.04%, 10.73% 	 -10.93% 	 0.98 avg gain/loss, -0.52 EV All, Buy and Hold 15 bars
45.54%, 14.08% 	 -14.67% 	 0.96 avg gain/loss, -1.58 EV All, Buy and Hold 30 bars
44.82%, 18.24% 	 -18.96% 	 0.96 avg gain/loss, -2.29 EV All, Buy and Hold 60 bars
15.19%, 32.58% 	 -7.70% 	 4.23 avg gain/loss, -1.58 EV All, Buy and Hold with Stop Loss of 5.00%
47.19%, 7.44% 	 -5.75% 	 1.29 avg gain/loss, 0.47 EV All, Buy and Hold 5 bars with Stop Loss of 5.00%
38.70%, 10.97% 	 -7.01% 	 1.57 avg gain/loss, -0.05 EV All, Buy and Hold 15 bars with Stop Loss of 5.00%
31.06%, 14.69% 	 -7.40% 	 1.98 avg gain/loss, -0.54 EV All, Buy and Hold 30 bars with Stop Loss of 5.00%
23.80%, 19.72% 	 -7.58% 	 2.60 avg gain/loss, -1.08 EV All, Buy and Hold 60 bars with Stop Loss of 5.00%
New Highs [8692]
49.00%, 23.13% 	 -20.78% 	 1.11 avg gain/loss, 0.74 EV New 

In [60]:
tradeSummaries |> List.filter (fun t -> t.EV > 0) |> describeOutcomes false

All [22848]
50.28%, 7.40% 	 -6.87% 	 1.08 avg gain/loss, 0.30 EV All, Buy and Hold 5 bars
47.19%, 7.44% 	 -5.75% 	 1.29 avg gain/loss, 0.47 EV All, Buy and Hold 5 bars with Stop Loss of 5.00%
New Highs [8692]
49.00%, 23.13% 	 -20.78% 	 1.11 avg gain/loss, 0.74 EV New Highs, Buy and Hold
Top Gainers [14156]
51.72%, 9.01% 	 -8.45% 	 1.07 avg gain/loss, 0.58 EV Top Gainers, Buy and Hold 5 bars
47.41%, 9.17% 	 -6.68% 	 1.37 avg gain/loss, 0.84 EV Top Gainers, Buy and Hold 5 bars with Stop Loss of 5.00%
36.73%, 13.78% 	 -7.70% 	 1.79 avg gain/loss, 0.19 EV Top Gainers, Buy and Hold 15 bars with Stop Loss of 5.00%
Gap ups [6447]
50.71%, 6.86% 	 -6.09% 	 1.13 avg gain/loss, 0.48 EV Gap ups, Buy and Hold 5 bars
49.74%, 9.76% 	 -9.52% 	 1.02 avg gain/loss, 0.07 EV Gap ups, Buy and Hold 15 bars
48.35%, 6.89% 	 -5.27% 	 1.31 avg gain/loss, 0.61 EV Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%
41.37%, 10.05% 	 -6.48% 	 1.55 avg gain/loss, 0.35 EV Gap ups, Buy and Hold 15 bars with Stop Loss


input.fsx (1,48)-(1,49) typecheck warning This expression uses the implicit conversion 'Decimal.op_Implicit(value: int) : decimal' to convert type 'int' to type 'decimal'. See https://aka.ms/fsharp-implicit-convs. This warning may be disabled using '#nowarn "3391".



In [25]:
tradeSummaries |> highlightStrategies 10

Top 10 by: EV
Gap ups - new highs, Buy and Hold: 1.822461023902737286555059309
Top Gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.8367691275013426500254850525
New Highs, Buy and Hold: 0.736784720611110804648903637
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.7342425861245473845493995603
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.6082295648889924875236600645
Top Gainers, Buy and Hold 5 bars: 0.5825484022408140426570786786
Gap ups - top gainers, Buy and Hold 5 bars: 0.5698105391586863628178409871
Gap ups - top gainers, Buy and Hold 15 bars with Stop Loss of 5.00%: 0.5252717112219428407362557718
Gap ups, Buy and Hold 5 bars: 0.4783539464164832646621860451
All, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4739211513203939924854011225


Top 10 by: Win %
Top Gainers, Buy and Hold 5 bars: 0.5171658660638598474145238768
Gap ups - top gainers, Buy and Hold 5 bars: 0.5154989384288747346072186837
Gap ups, Buy and Hold 5 bars: 0.50705754614549402823

In [26]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Types.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.Total)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    printfn $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    printfn $"mapped keys and values for {label}, charting..."
    
    values |> List.zip keys |> List.iter (fun (l,v) -> printfn $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    printfn ""
)

sorting for EV...
mapped keys and values for EV, charting...
Gap ups - new highs, Buy and Hold: 1.822461023902737286555059309
Top Gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.8367691275013426500254850525
New Highs, Buy and Hold: 0.736784720611110804648903637
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.7342425861245473845493995603
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.6082295648889924875236600645
Top Gainers, Buy and Hold 5 bars: 0.5825484022408140426570786786
Gap ups - top gainers, Buy and Hold 5 bars: 0.5698105391586863628178409871
Gap ups - top gainers, Buy and Hold 15 bars with Stop Loss of 5.00%: 0.5252717112219428407362557718
Gap ups, Buy and Hold 5 bars: 0.4783539464164832646621860451
All, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4739211513203939924854011225
Gap ups, Buy and Hold 15 bars with Stop Loss of 5.00%: 0.3545739410194513842326845869
All, Buy and Hold 5 bars: 0.3049850560031265735744391013
Gap ups - top gainers

<!-- Plotly chart will be drawn inside this DIV -->


sorting for Win/Loss...
mapped keys and values for Win/Loss, charting...
Top Gainers, Buy and Hold with Stop Loss of 5.00%: 4.6520815047384464540047606012
Gap ups - top gainers, Buy and Hold with Stop Loss of 5.00%: 4.4009138806675472675252545609
Gap ups, Buy and Hold with Stop Loss of 5.00%: 4.2616818470896314596810542204
All, Buy and Hold with Stop Loss of 5.00%: 4.2304836883926561831266856358
Gap ups - new highs, Buy and Hold with Stop Loss of 5.00%: 3.9570277970845509699086760666
New Highs, Buy and Hold with Stop Loss of 5.00%: 3.741701936678974576469315427
Top Gainers, Buy and Hold 60 bars with Stop Loss of 5.00%: 2.9779853467285827701055597768
Gap ups - top gainers, Buy and Hold 60 bars with Stop Loss of 5.00%: 2.7972043794092182793608283922
Gap ups, Buy and Hold 60 bars with Stop Loss of 5.00%: 2.6234192715406586846959879479
All, Buy and Hold 60 bars with Stop Loss of 5.00%: 2.6007702858143031990484481471
Top Gainers, Buy and Hold 30 bars with Stop Loss of 5.00%: 2.291822051375

<!-- Plotly chart will be drawn inside this DIV -->


sorting for Win %...
mapped keys and values for Win %, charting...
Top Gainers, Buy and Hold 5 bars: 0.5171658660638598474145238768
Gap ups - top gainers, Buy and Hold 5 bars: 0.5154989384288747346072186837
Gap ups, Buy and Hold 5 bars: 0.507057546145494028230184582
All, Buy and Hold 5 bars: 0.5028448879551820728291316527
Gap ups - top gainers, Buy and Hold 15 bars: 0.5016985138004246284501061571
Gap ups, Buy and Hold 15 bars: 0.497440670079106561191251745
Gap ups - top gainers, Buy and Hold 60 bars: 0.4912951167728237791932059448
New Highs, Buy and Hold: 0.4899907961343764381040036815
Gap ups - new highs, Buy and Hold 15 bars: 0.4858952216465169833045480714
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4855626326963906581740976645
Gap ups - new highs, Buy and Hold 5 bars: 0.4841681059297639608520437536
Gap ups - new highs, Buy and Hold: 0.4841681059297639608520437536
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: 0.4834806886924150767798976268
Gap ups, 

...


sorting for Avg Loss...
mapped keys and values for Avg Loss, charting...
Gap ups - new highs, Buy and Hold 5 bars with Stop Loss of 5.00%: -3.9758394091644479445196008064
Gap ups - new highs, Buy and Hold 5 bars: -4.1562593719127638953214441605
New Highs, Buy and Hold 5 bars with Stop Loss of 5.00%: -4.2435446137399884640129785248
New Highs, Buy and Hold 5 bars: -4.4953763690384411117444266849
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%: -5.2681274300914848142805532396
Gap ups - new highs, Buy and Hold 15 bars with Stop Loss of 5.00%: -5.485425623325706640651843268
All, Buy and Hold 5 bars with Stop Loss of 5.00%: -5.7470584161502600781671790251
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: -5.7512657141766962825451938474
New Highs, Buy and Hold 15 bars with Stop Loss of 5.00%: -5.792178707077294940211943247
Gap ups, Buy and Hold 5 bars: -6.0853413925882840043377146294
Gap ups - new highs, Buy and Hold 30 bars with Stop Loss of 5.00%: -6.2755302365584448

<!-- Plotly chart will be drawn inside this DIV -->


sorting for Number of Trades...
mapped keys and values for Number of Trades, charting...
All, Buy and Hold 5 bars: 22848
All, Buy and Hold 15 bars: 22848
All, Buy and Hold 30 bars: 22848
All, Buy and Hold 60 bars: 22848
All, Buy and Hold: 22848
All, Buy and Hold 5 bars with Stop Loss of 5.00%: 22848
All, Buy and Hold 15 bars with Stop Loss of 5.00%: 22848
All, Buy and Hold 30 bars with Stop Loss of 5.00%: 22848
All, Buy and Hold 60 bars with Stop Loss of 5.00%: 22848
All, Buy and Hold with Stop Loss of 5.00%: 22848
Top Gainers, Buy and Hold 5 bars: 14156
Top Gainers, Buy and Hold 15 bars: 14156
Top Gainers, Buy and Hold 30 bars: 14156
Top Gainers, Buy and Hold 60 bars: 14156
Top Gainers, Buy and Hold: 14156
Top Gainers, Buy and Hold 5 bars with Stop Loss of 5.00%: 14156
Top Gainers, Buy and Hold 15 bars with Stop Loss of 5.00%: 14156
Top Gainers, Buy and Hold 30 bars with Stop Loss of 5.00%: 14156
Top Gainers, Buy and Hold 60 bars with Stop Loss of 5.00%: 14156
Top Gainers, Buy and Ho

<!-- Plotly chart will be drawn inside this DIV -->


input.fsx (9,35)-(9,42) typecheck warning This expression uses the implicit conversion 'Decimal.op_Implicit(value: int) : decimal' to convert type 'int' to type 'decimal'. See https://aka.ms/fsharp-implicit-convs. This warning may be disabled using '#nowarn "3391".



In [43]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    printfn "%.2f%%, %.2f%% \t %.2f%% \t %.2f avg gain/loss, %.2f EV %s" 
        (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV summary.StrategyName
        
tradeSummaries
|> List.filter ( fun s -> s.StrategyName.Contains("Gap ups - new highs, Buy and Hold"))
|> List.sortByDescending ( fun s -> s.EV)
|> List.iter (fun s -> s |> outputSummary false)

48.42%, 24.00% 	 -18.99% 	 1.26 avg gain/loss, 1.82 EV Gap ups - new highs, Buy and Hold
47.78%, 4.90% 	 -3.98% 	 1.23 avg gain/loss, 0.27 EV Gap ups - new highs, Buy and Hold 5 bars with Stop Loss of 5.00%
48.42%, 4.91% 	 -4.16% 	 1.18 avg gain/loss, 0.23 EV Gap ups - new highs, Buy and Hold 5 bars
20.21%, 26.98% 	 -6.82% 	 3.96 avg gain/loss, 0.01 EV Gap ups - new highs, Buy and Hold with Stop Loss of 5.00%
43.47%, 6.89% 	 -5.49% 	 1.26 avg gain/loss, -0.11 EV Gap ups - new highs, Buy and Hold 15 bars with Stop Loss of 5.00%
36.61%, 10.06% 	 -6.28% 	 1.60 avg gain/loss, -0.29 EV Gap ups - new highs, Buy and Hold 30 bars with Stop Loss of 5.00%
48.59%, 6.74% 	 -6.96% 	 0.97 avg gain/loss, -0.30 EV Gap ups - new highs, Buy and Hold 15 bars
46.69%, 9.78% 	 -9.38% 	 1.04 avg gain/loss, -0.43 EV Gap ups - new highs, Buy and Hold 30 bars
28.09%, 14.14% 	 -6.63% 	 2.13 avg gain/loss, -0.80 EV Gap ups - new highs, Buy and Hold 60 bars with Stop Loss of 5.00%
45.60%, 12.39% 	 -13.17% 	 0.94 a

In [35]:
// let's take only the trades where EV is positive
tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary false s)

Gap ups - new highs, Buy and Hold:
 1737 trades, 48.42% win pct, 24.00% avg gain, -18.99% avg loss, 1.26 avg gain/loss, 1.82 EV
Top Gainers, Buy and Hold 5 bars with Stop Loss of 5.00%:
 14156 trades, 47.41% win pct, 9.17% avg gain, -6.68% avg loss, 1.37 avg gain/loss, 0.84 EV
New Highs, Buy and Hold:
 8692 trades, 49.00% win pct, 23.13% avg gain, -20.78% avg loss, 1.11 avg gain/loss, 0.74 EV
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%:
 4710 trades, 48.56% win pct, 7.61% avg gain, -5.75% avg loss, 1.32 avg gain/loss, 0.73 EV
Gap ups, Buy and Hold 5 bars with Stop Loss of 5.00%:
 6447 trades, 48.35% win pct, 6.89% avg gain, -5.27% avg loss, 1.31 avg gain/loss, 0.61 EV
Top Gainers, Buy and Hold 5 bars:
 14156 trades, 51.72% win pct, 9.01% avg gain, -8.45% avg loss, 1.07 avg gain/loss, 0.58 EV
Gap ups - top gainers, Buy and Hold 5 bars:
 4710 trades, 51.55% win pct, 7.54% avg gain, -6.84% avg loss, 1.10 avg gain/loss, 0.57 EV
Gap ups - top gainers, Buy and Hold 15 